##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet                                                                                                                    # IGNORE_COPYRIGHT: cleared by OSS licensing
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 미리 학습된 CNN 모델로 Transfer learning

In this tutorial, you will learn how to classify images of cats and dogs by using transfer learning from a pre-trained network.

이미 큰 데이터셋으로 학습된 CNN 모델의 feature를 사용하여 고양이와 강아지를 분류하는 모델을 fine-tuning
- 이미 잘 학습된 filter 값들을 사용하여 feature map들을 만들어내기 때문에, 빠르게 학습할 수 있다는 장점이 있음
- frozen 모델의 상위 layer 몇 개만 새롭게 추가된 분류 layer와 함께 학습을 시키고, 나머지 layer들은 고정된 채로 학습에 참여

다음과 같은 순서로 진행
1. 데이터셋 파악
1. Keras ImageDataGenerator를 이용한 pipeline 제작
1. 모델 구성
   * pretrained base model 및 weights 불러오기
   * 분류를 위한 layer 추가
1. 모델 학습
1. 모델 평가


In [ ]:
import os

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf


## 전처리

### 데이터 다운로드

[TensorFlow Datasets](http://tensorflow.org/datasets) 으로 부터 고양이와 강아지 데이터셋 로드


In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

`"cats_vs_dogs"` 데이터셋은 정해진 training / validation 분리 비율이 없기 때문에, 임의로 (train, validation, test)를 (80%, 10%, 10%)로 분리

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

RGB 3 채널과 데이터 type이 int임을 확인 할 수 있음

In [ ]:
print(raw_train)
print(raw_validation)
print(raw_test)

학습 데이터셋의 첫 두 이미지와 정답을 출력

In [ ]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

### 데이터 formatting

In [ ]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

map 함수를 사용하여 위의 전처리 함수를 적용

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

batch 구성 & shuffle

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

## pre-trained CNN으로부터 base 모델 생성
You will create the base model from the **MobileNet V2** model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset consisting of 1.4M images and 1000 classes. ImageNet is a research training dataset with a wide variety of categories like `jackfruit` and `syringe`. This base of knowledge will help us classify cats and dogs from our specific dataset.

Google의 **MobileNet V2**으로부터 base 모델 생성
ImageNet dataset으로 미리 학습된 모델 (1.4M 이미지 & 1000 classes)
ImageNet으로 학습된 parameter들이 고양이&강아지를 분류하는데 도움을 줌

feature extraction을 위해 사용할 MobileNet V2의 layer를 선정
- 마지막 layer는 ImageNet의 분류를 위한 layer이기 때문에 도움이 안 됨
- Flatten operation 이전의 layer를 일반적으로 사용 (bottleneck layer 라고 함)
- **include_top=False** argument을 설정함으로써, 마지막 분류를 위한 layer를 제외한 모델을 불러올 수 있음

First, you need to pick which layer of MobileNet V2 you will use for feature extraction. The very last classification layer (on "top", as most diagrams of machine learning models go from bottom to top) is not very useful.  Instead, you will follow the common practice to depend on the very last layer before the flatten operation. This layer is called the "bottleneck layer". The bottleneck layer features retain more generality as compared to the final/top layer.

First, instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the **include_top=False** argument, you load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Feature extractor는 `160x160x3` 이미지를 `5x5x1280` 의 feature로 변환

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Feature extraction

### Freeze the convolutional base

모델을 학습시키기 전에 base CNN 모델을 freeze 시켜놓는 것이 필요
- base CNN 모델의 학습 parameter가 update 되지 않도록

In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Classification head 추가

이미지당 1280-element vector를 만들어 낼 수 있도록 `tf.keras.layers.GlobalAveragePooling2D` layer를 사용하여 `5x5` 마다 평균내는 pooling 사용

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

분류를 위한 `tf.keras.layers.Dense` layer를 추가

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

`tf.keras.Sequential` 을 사용하여 base CNN 모델 (feature extractor) 위에 2개의 layer를 추가

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

### 모델 컴파일

모델이 logit 값을 output으로 갖기 때문에 `from_logits=True` argument 세팅

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
model.summary()

약 200만개의 parameter가 학습 되지 않고, 학습되는 parameter는 오직 마지막 분류를 위한 Dense layer에만 존재

In [ ]:
len(model.trainable_variables)

### 모델 학습

10 epoch 학습 후 ~96% 정확도를 보임

1 epoch만 학습하여 test


In [ ]:
initial_epochs = 2
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

### 학습 curve 확인

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

학습에 `tf.keras.layers.BatchNormalization`, `tf.keras.layers.Dropout`이 사용되기 때문에 validation metric이 training metric 보다 더 좋은 성능을 기록할 수 있음

또한, validation은 학습이 진행된 후 측정되는 것이기 때문에, training 때보다 좋은 성능을 기록할 수 있음

## Fine tuning

모델의 성능을 향상시키는 다른 방법: pre-trained base CNN 모델의 최상위 layer를 학습에 참여시킴

학습이 진행되면서 이미 학습되었던 weight 들이 우리의 task에 맞게 loss가 작게 생성되도록 변화해 나감

단, 분류를 위한 layer를 먼저 학습시키고 난 후, 학습 가능하도록 전환해야함
(random하게 초기화되어 있는 분류 layer와 처음부터 같이 학습해버리면, gradient의 값이 엄청 크게 나오기 때문에 ImageNet으로 학습했던 것들을 잊어버림)

또한, fine-tune에 사용되는 상위 layer는 전체 MobileNet base 모델 대비 소수여야 함
- 일반적으로 CNN 모델의 상위 layer는 task를 위해 특화가 되어지고, 하위 layer는 일반적인 이미지의 특징을 잡기 위한 방향으로 학습이 되어지기 때문에 이 부분은 살려야 함


### base CNN 모델의 상위 레이어 parameter un-freeze


Base CNN 모델의 상위 레이어들을 학습 가능한 상태로 바꾸고, 하위 레이어들은 학습이 불가능 하도록 세팅하여 recompile

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### 모델 컴파일

작은 learning rate 사용 (이미 학습이 많이 진행된 모델이기 때문에)

In [ ]:
model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### 모델 학습

In [ ]:
fine_tune_epochs = 2
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

학습을 많이 진행한 후 학습 curve를 살펴보면, validation loss가 training loss보다 훨씬 큰 overfitting 현상을 발견할 수 있음
(이미 비슷한 성격의 ImageNet 데이터셋으로 충분히 학습을 시킨 상태이고, 새로운 데이터셋이 ImageNet에 비하면 상대적으로 작기 때문)


In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 요약:

* **Feature extraction을 위한 미리 학습된 모델 사용**:  
작은 데이터셋으로 학습을 진행할 때, 큰 데이터셋으로 미리 학습해놓은 모델의 feature를 사용하는 것이 일반적
  - pre-trained 모델을 불러와서 분류를 위한 layer를 제외한 나머지 layer의 parameter를 학습 불가능 상태(freeze)로 바꿔놓고, 분류를 위한 layer만 학습 진행

* **Fine-tuning**: 
모델의 성능을 더욱 향상시키기 위해서 pre-trained base 모델의 상위 layer 들 일부를 학습 가능상태로 전환
  - 새로 학습시키는 데이터셋이 크고, base 모델을 학습시킨 데이터셋과 비슷할 경우 권장됨
